# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [25]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)
sum = 0
total = 0
for sqrt_value in generator:
    print(sqrt_value)
    sum = sqrt_value
    total+=sum
print(f"sum of the outputs of the generator for limit = {limit} is :", total)

print("\n\n\n")
# Example usage:
limit = 13
generator = square_root_generator(limit)
sum = 0

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
sum of the outputs of the generator for limit = 5 is : 8.382332347441762




1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [4]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [5]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [ ]:
# to do: homework :)

In [15]:
import dlt

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='homeworkgenerators')

# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="http_download",
										write_disposition="replace")

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.20 seconds
1 load package(s) were loaded to destination duckdb and into dataset homeworkgenerators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707281399.1767728 is LOADED and contains no failed jobs


In [16]:
# show outcome

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

print("\n\n\n http_download table below:")

people1 = conn.sql("SELECT * FROM http_download").df()
display(people1)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ http_download       │
└─────────────────────┘




 http_download table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707281399.1767728,unHUdnZJzLHg7Q,None
1,2,Person_2,27,City_A,1707281399.1767728,LZPtVZBpGqkJew,None
2,3,Person_3,28,City_A,1707281399.1767728,J7BktYFlwBfRSA,None
3,4,Person_4,29,City_A,1707281399.1767728,DnHT/if6Kk1KGw,None
4,5,Person_5,30,City_A,1707281399.1767728,dgb/Qarj7X0pEg,None


In [17]:
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='homeworkgenerators')

# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_2(),
										table_name="http_download",
										write_disposition="merge",
                    primary_key="city")

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.31 seconds
1 load package(s) were loaded to destination duckdb and into dataset homeworkgenerators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707281428.4197178 is LOADED and contains no failed jobs


In [18]:
# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

print("\n\n\n http_download table below:")

people1 = conn.sql("SELECT * FROM http_download").df()
display(people1)

# show outcome

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

print("\n\n\n http_download table below:")

people1 = conn.sql("SELECT * FROM http_download").df()
display(people1)




Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ http_download       │
└─────────────────────┘




 http_download table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707281399.1767728,unHUdnZJzLHg7Q,None
1,2,Person_2,27,City_A,1707281399.1767728,LZPtVZBpGqkJew,None
2,3,Person_3,28,City_A,1707281399.1767728,J7BktYFlwBfRSA,None
3,4,Person_4,29,City_A,1707281399.1767728,DnHT/if6Kk1KGw,None
4,5,Person_5,30,City_A,1707281399.1767728,dgb/Qarj7X0pEg,None
5,6,Person_6,36,City_B,1707281428.4197178,QD/4jNU0odSwmQ,Job_6
6,7,Person_7,37,City_B,1707281428.4197178,JCCsT5lzPTsJtw,Job_7
7,3,Person_3,33,City_B,1707281428.4197178,mypNqj6jDv1n3g,Job_3
8,4,Person_4,34,City_B,1707281428.4197178,ULPppS6mAmMaHw,Job_4
9,5,Person_5,35,City_B,1707281428.4197178,ogijFjHhlZNg6A,Job_5


In [19]:
# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

print("\n\n\n http_download table below:")

people1 = conn.sql("SELECT SUM(age) FROM http_download").df()
display(people1)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ http_download       │
└─────────────────────┘




 http_download table below:


,sum(age)
0,353.0


In [23]:
import dlt

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='sqrtgenerators')

# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="sqrt_gen",
										write_disposition="replace")

print(info)

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

print("\n\n\n http_download table below:")

people1 = conn.sql("SELECT * FROM sqrt_gen").df()
display(people1)

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='sqrtgenerators')

# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_2(),
										table_name="sqrt_gen",
										write_disposition="append")

print(info)


import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

print("\n\n\n http_download table below:")

people1 = conn.sql("SELECT * FROM sqrt_gen").df()
display(people1)
print("\n\n\n")
age_sum = conn.sql("SELECT SUM(age) FROM sqrt_gen").df()
display(age_sum)

Pipeline dlt_colab_kernel_launcher load step completed in 0.20 seconds
1 load package(s) were loaded to destination duckdb and into dataset sqrtgenerators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707282550.7425234 is LOADED and contains no failed jobs
Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ sqrt_gen            │
└─────────────────────┘




 http_download table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707282550.7425234,md3MISuhT1heOA,None
1,2,Person_2,27,City_A,1707282550.7425234,aHkB+tD/yw1yyQ,None
2,3,Person_3,28,City_A,1707282550.7425234,AstFCWLKC0yifw,None
3,4,Person_4,29,City_A,1707282550.7425234,BhTceJKTs0S2xQ,None
4,5,Person_5,30,City_A,1707282550.7425234,N0Mrx8yr0igiMg,None


Pipeline dlt_colab_kernel_launcher load step completed in 0.21 seconds
1 load package(s) were loaded to destination duckdb and into dataset sqrtgenerators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707282551.3895304 is LOADED and contains no failed jobs
Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ sqrt_gen            │
└─────────────────────┘




 http_download table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707282550.7425234,md3MISuhT1heOA,None
1,2,Person_2,27,City_A,1707282550.7425234,aHkB+tD/yw1yyQ,None
2,3,Person_3,28,City_A,1707282550.7425234,AstFCWLKC0yifw,None
3,4,Person_4,29,City_A,1707282550.7425234,BhTceJKTs0S2xQ,None
4,5,Person_5,30,City_A,1707282550.7425234,N0Mrx8yr0igiMg,None
5,3,Person_3,33,City_B,1707282551.3895304,raL6gwlbQXARzw,Job_3
6,4,Person_4,34,City_B,1707282551.3895304,FweIGGaeAmTg0Q,Job_4
7,5,Person_5,35,City_B,1707282551.3895304,V+cvZ30VQywuVQ,Job_5
8,6,Person_6,36,City_B,1707282551.3895304,qQE3eefSQxwZcQ,Job_6
9,7,Person_7,37,City_B,1707282551.3895304,TYukXkJnrZ0mhQ,Job_7


,sum(age)
0,353.0


In [ ]:
# merge

In [32]:
import dlt

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='sqrtgenerators')

# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="sqrt_gen",
										write_disposition="replace")

print(info)

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

print("\n\n\n http_download table below:")

people1 = conn.sql("SELECT * FROM sqrt_gen").df()
display(people1)

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='sqrtgenerators')

# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_2(),
										table_name="sqrt_gen",
										write_disposition="merge",
                    primary_key="id")

print(info)


import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

print("\n\n\n http_download table below:")

people1 = conn.sql("SELECT * FROM sqrt_gen").df()
display(people1)
print("\n\n\n")
age_sum = conn.sql("SELECT SUM(age) FROM sqrt_gen").df()
display(age_sum)

Pipeline dlt_colab_kernel_launcher load step completed in 0.26 seconds
1 load package(s) were loaded to destination duckdb and into dataset sqrtgenerators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707284288.703177 is LOADED and contains no failed jobs
Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ sqrt_gen            │
└─────────────────────┘




 http_download table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707284288.703177,cZHsYHIdwa3fQg,None
1,2,Person_2,27,City_A,1707284288.703177,OgcZ3ycRxwpYNA,None
2,3,Person_3,28,City_A,1707284288.703177,OMgZdlt2wGTbzA,None
3,4,Person_4,29,City_A,1707284288.703177,bSvNZi24TNhdWg,None
4,5,Person_5,30,City_A,1707284288.703177,XA/xWsFAtliBwQ,None


Pipeline dlt_colab_kernel_launcher load step completed in 0.46 seconds
1 load package(s) were loaded to destination duckdb and into dataset sqrtgenerators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707284289.504588 is LOADED and contains no failed jobs
Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ sqrt_gen            │
└─────────────────────┘




 http_download table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707284288.703177,cZHsYHIdwa3fQg,None
1,2,Person_2,27,City_A,1707284288.703177,OgcZ3ycRxwpYNA,None
2,8,Person_8,38,City_B,1707284289.504588,P7In14nW4nXoMg,Job_8
3,7,Person_7,37,City_B,1707284289.504588,BINUapLEHehb7A,Job_7
4,5,Person_5,35,City_B,1707284289.504588,usaNa8XK97U2MA,Job_5
5,4,Person_4,34,City_B,1707284289.504588,mNI07iwWef69wA,Job_4
6,3,Person_3,33,City_B,1707284289.504588,YMRwdzYY6Gu7tQ,Job_3
7,6,Person_6,36,City_B,1707284289.504588,HLH1dzXzVftMAw,Job_6


,sum(age)
0,266.0


# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

print("\n\n\n http_download table below:")

people1 = conn.sql("SELECT SUM("age") FROM http_download").df()
display(people1)

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX